In [1]:
from tabulate import tabulate

In [2]:
def dot_matrix(a, b, size, step, thresh):
    dot_arr = [[' ']* (len(a)+1) for i in range(len(b) + 1)] 
    for i in range(1, len(a) + 1):
        dot_arr[0][i] = a[i-1]

    for i in range(1, len(b) + 1):
        dot_arr[i][0] = b[i-1]

    def thresh_check(a, b): 
        count = 0
        for i,j in zip(a,b):
            if i == j:
                count += 1
        if count >= thresh: 
            return True

    for i in range((((len(a) * (size - step))//size) + len(a))//size):
        for j in range((((len(b) * (size - step))//size) + len(b))//size):
            if thresh_check(a[i*step: i*step + size],b[j*step: j*step + size]): 
                dot_arr[j*step + size//2 + 1][i*step + size//2 + 1] = 'x'

    print(tabulate(dot_arr, tablefmt="github"))

In [3]:

def needleman(a, b, match, mismatch, gap):
    score = 0
    dot_arr = [[' '] * (len(a) + 2) for i in range(len(b) + 2)]
    dot_arr[0][1] = 'i'
    dot_arr[1][0] = 'j'
    dot_arr[1][1] = 0

    for i in range(2, len(a) + 2):
        dot_arr[0][i] = a[i-2]
        dot_arr[1][i] = dot_arr[1][i-1] + gap

    for i in range(2, len(b) + 2):
        dot_arr[i][0] = b[i-2]
        dot_arr[i][1] = dot_arr[i-1][1] + gap

    def p(alligns):
        cnt = 1
        for i in alligns:   
            print(f'\n({cnt})\n')
            for j in i:
                print(j)
            cnt += 1

    #Traceback Function   
    def traceback(seq1, seq2, dot_arr, col, row):
        allign = []
        if col == 1 and row > 1:
            return [['-' * len(seq2[:row - 1]), seq2[:row - 1]]]
        if col > 1 and row == 1: 
            return [[seq1[:col - 1], '-' * len(seq1[:col - 1])]]
        if col == 1 and row == 1: 
            return [["", ""]]

        if seq1[col-2] == seq2[row-2]:
            vals = traceback(seq1, seq2, dot_arr, col - 1, row - 1)
            for value in vals:
                value[0] += seq1[col-2]
                value[1] += seq1[col-2]
            allign += vals
        else:
            up = dot_arr[row - 1][col] 
            left = dot_arr[row][col - 1]
            diagonal = dot_arr[row - 1][col - 1]
            maxi = max(up,left,diagonal)
            if maxi == up:
                vals = traceback(seq1, seq2, dot_arr, col, row - 1)
                for value in vals:
                    value[0] += '-'
                    value[1] += seq2[row - 2]
                allign += vals
            if maxi == left:
                vals = traceback(seq1, seq2, dot_arr, col - 1, row)
                for value in vals:
                    value[0] += seq1[col - 2]
                    value[1] += '-'
                allign += vals
            if maxi == diagonal: 
                vals = traceback(seq1, seq2, dot_arr, col - 1, row - 1)
                for value in vals:
                    value[0] += seq1[col - 2]
                    value[1] += seq2[row - 2]
                allign += vals
        return allign


    for i in range(2, len(a) + 2):
        for j in range(2, len(b) + 2):
            up = dot_arr[j-1][i] + gap
            left = dot_arr[j][i-1] + gap
            if dot_arr[j][0] == dot_arr[0][i]:
                diagonal = dot_arr[j-1][i-1] + match
            else:
                diagonal = dot_arr[j-1][i-1] + mismatch
            
            dot_arr[j][i] = max(up, left, diagonal)


    print(tabulate(dot_arr, tablefmt="github"))
    print("\nTraceback: ")
    #print(traceback(a, b, dot_arr, len(a) + 1, len(b) + 1))
    final = traceback(a, b, dot_arr, len(a) + 1, len(b) + 1)
    p(final)



In [4]:

def waterman(a, b, match, mismatch, gap):
    score = 0
    dot_arr = [[' '] * (len(a) + 2) for i in range(len(b) + 2)]
    dot_arr[0][1] = 'i'
    dot_arr[1][0] = 'j'
    dot_arr[1][1] = 0

    def p(alligns):
        cnt = 1
        for i in alligns:   
            print(f'\n({cnt})\n')
            for j in i:
                print(j)
            cnt += 1

    def traceback(seq1, seq2, dot_arr, col, row):
        allign = []
        if dot_arr[row][col] == 0: 
            return [["", ""]]

        if seq1[col-2] == seq2[row-2]:
            vals = traceback(seq1, seq2, dot_arr, col - 1, row - 1)
            for value in vals:
                value[0] += seq1[col-2]
                value[1] += seq1[col-2]
            allign += vals
        else:
            up = dot_arr[row - 1][col] 
            left = dot_arr[row][col - 1]
            diagonal = dot_arr[row - 1][col - 1]
            maxi = max(up,left,diagonal)
            if maxi == up:
                vals = traceback(seq1, seq2, dot_arr, col, row - 1)
                for value in vals:
                    value[0] += '-'
                    value[1] += seq2[row - 2]
                allign += vals
            if maxi == left:
                vals = traceback(seq1, seq2, dot_arr, col - 1, row)
                for value in vals:
                    value[0] += seq1[col - 2]
                    value[1] += '-'
                allign += vals
            if maxi == diagonal: 
                vals = traceback(seq1, seq2, dot_arr, col - 1, row - 1)
                for value in vals:
                    value[0] += seq1[col - 2]
                    value[1] += seq2[row - 2]
                allign += vals
        return allign


    for i in range(2, len(a) + 2):
        dot_arr[0][i] = a[i-2]
        dot_arr[1][i] = 0

    for i in range(2, len(b) + 2):
        dot_arr[i][0] = b[i-2]
        dot_arr[i][1] = 0

    max_value = 0
    mx = 0
    my = 0

    for i in range(2, len(a) + 2):
        for j in range(2, len(b) + 2):
            up = dot_arr[j-1][i] + gap
            left = dot_arr[j][i-1] + gap
            if dot_arr[j][0] == dot_arr[0][i]:
                diagonal = dot_arr[j-1][i-1] + match
            else:
                diagonal = dot_arr[j-1][i-1] + mismatch
            dot_arr[j][i] = max(up, left, diagonal, 0)
            if max_value <= dot_arr[j][i]:
                max_value = dot_arr[j][i]
                mx = j
                my = i

    print(tabulate(dot_arr, tablefmt="github"))
    final = traceback(a, b, dot_arr, my, mx)
    p(final)



In [5]:
needleman(a = "ACGTTGACCTGTAACCTC" , b = "ACCTTGTCCTCTTTGCCC", match = 2, mismatch=-1, gap = -1)

|---|-----|-----|-----|----|----|----|----|----|----|----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|   | i   | A   | C   | G  | T  | T  | G  | A  | C  | C  | T   | G   | T   | A   | A   | C   | C   | T   | C   |
| j | 0   | -1  | -2  | -3 | -4 | -5 | -6 | -7 | -8 | -9 | -10 | -11 | -12 | -13 | -14 | -15 | -16 | -17 | -18 |
| A | -1  | 2   | 1   | 0  | -1 | -2 | -3 | -4 | -5 | -6 | -7  | -8  | -9  | -10 | -11 | -12 | -13 | -14 | -15 |
| C | -2  | 1   | 4   | 3  | 2  | 1  | 0  | -1 | -2 | -3 | -4  | -5  | -6  | -7  | -8  | -9  | -10 | -11 | -12 |
| C | -3  | 0   | 3   | 3  | 2  | 1  | 0  | -1 | 1  | 0  | -1  | -2  | -3  | -4  | -5  | -6  | -7  | -8  | -9  |
| T | -4  | -1  | 2   | 2  | 5  | 4  | 3  | 2  | 1  | 0  | 2   | 1   | 0   | -1  | -2  | -3  | -4  | -5  | -6  |
| T | -5  | -2  | 1   | 1  | 4  | 7  | 6  | 5  | 4  | 3  | 2   | 1   | 3   | 2   | 1   | 0   | -1  | -2  | -3  |
| G | -6  | -3  | 0   | 3  | 3  | 6  | 9  | 8  | 7  | 6  | 5   | 4   | 3   | 2   | 1   | 0   | -

In [6]:
waterman(a = "ACGTTGACCTGTAACCTC" , b = "ACCTTGTCCTCTTTGCCC", match = 2, mismatch=-1, gap = -1)

|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|----|----|----|----|----|
|   | i | A | C | G | T | T | G | A | C  | C  | T  | G  | T  | A  | A  | C  | C  | T  | C  |
| j | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| A | 0 | 2 | 1 | 0 | 0 | 0 | 0 | 2 | 1  | 0  | 0  | 0  | 0  | 2  | 2  | 1  | 0  | 0  | 0  |
| C | 0 | 1 | 4 | 3 | 2 | 1 | 0 | 1 | 4  | 3  | 2  | 1  | 0  | 1  | 1  | 4  | 3  | 2  | 2  |
| C | 0 | 0 | 3 | 3 | 2 | 1 | 0 | 0 | 3  | 6  | 5  | 4  | 3  | 2  | 1  | 3  | 6  | 5  | 4  |
| T | 0 | 0 | 2 | 2 | 5 | 4 | 3 | 2 | 2  | 5  | 8  | 7  | 6  | 5  | 4  | 3  | 5  | 8  | 7  |
| T | 0 | 0 | 1 | 1 | 4 | 7 | 6 | 5 | 4  | 4  | 7  | 7  | 9  | 8  | 7  | 6  | 5  | 7  | 7  |
| G | 0 | 0 | 0 | 3 | 3 | 6 | 9 | 8 | 7  | 6  | 6  | 9  | 8  | 8  | 7  | 6  | 5  | 6  | 6  |
| T | 0 | 0 | 0 | 2 | 5 | 5 | 8 | 8 | 7  | 6  | 8  | 8  | 11 | 10 | 9  | 8  | 7  | 7  | 6  |
| C | 0 | 0 | 2 | 1 | 4 | 4 | 7 | 7 | 10 | 9  | 8  | 7  | 10 | 10 | 9 

In [7]:
dot_matrix(a = "ACGTAGATG", b = "ACGAAG", size = 3, step = 3, thresh = 2)

|---|---|---|---|---|---|---|---|---|---|
|   | A | C | G | T | A | G | A | T | G |
| A |   |   |   |   |   |   |   |   |   |
| C |   | x |   |   |   |   |   | x |   |
| G |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |
| A |   | x |   |   | x |   |   | x |   |
| G |   |   |   |   |   |   |   |   |   |
